In [4]:
import tensorflow as tf
from pathlib import Path
import numpy as np

In [5]:
data = np.load(Path('generated_samples') / "train" /  '00000.npz')
h, w, c = data["arr_0"].shape

In [6]:
gen_samples_path = Path("generated_samples")
def load_images(gen_samples_path, mode):
    def gen_img():
        for file in list((gen_samples_path / mode).iterdir()):
            data = np.load(file)
            
            yield data["arr_0"].astype(np.float32), data["arr_1"].astype(np.float32)
    return gen_img
#train_dataset = tf.data.Dataset.list_files(str(gen_samples_path / "train" / "*.npz"), shuffle=True)

train_dataset = tf.data.Dataset.from_generator(load_images(gen_samples_path, "train"), 
                                               output_signature= (tf.TensorSpec((h, w, c), dtype=tf.float32), 
                                                                  tf.TensorSpec((h, w, 2), dtype=tf.float32)))
validation_dataset = tf.data.Dataset.from_generator(load_images(gen_samples_path, "validation"), 
                                               output_signature= (tf.TensorSpec((h, w, c), dtype=tf.float32), 
                                                                  tf.TensorSpec((h, w, 2), dtype=tf.float32)))



In [7]:
from unet.callbacks import TensorBoardImageSummary
log_dir_path = "unet_lofar_run_1"
tb_img = TensorBoardImageSummary("validation", logdir=log_dir_path, dataset=validation_dataset, max_outputs=10)

In [8]:
import unet
from unet.datasets import circles

#building the model
unet_model = unet.build_model(channels=c,
                              num_classes=2,
                              layer_depth=2,
                              filters_root=16)

unet.finalize_model(unet_model)

#training and validating the model
trainer = unet.Trainer(name="unet_gen_test", log_dir_path ="unet_lofar_run_1", checkpoint_callback =True, tensorboard_callback=True)#, tensorboard_images_callback=tb_img)
trainer.fit(unet_model,
           train_dataset,
           validation_dataset,
           epochs=10,
           batch_size=64)

(143, 533, 1)
Epoch 1/10
1/1 [==============================] - 11s 11s/step - loss: 0.7194 - categorical_crossentropy: 0.7194 - categorical_accuracy: 0.2614 - mean_iou: 0.2571 - dice_coefficient: 0.4934 - auc: 0.4512 - val_loss: 0.6944 - val_categorical_crossentropy: 0.6944 - val_categorical_accuracy: 0.1961 - val_mean_iou: 0.2634 - val_dice_coefficient: 0.4995 - val_auc: 0.4757


INFO:tensorflow:Assets written to: unet_lofar_run_1/assets


INFO:tensorflow:Assets written to: unet_lofar_run_1/assets


Epoch 2/10


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pedro/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-7a9b7ff913cb>", line 14, in <module>
    trainer.fit(unet_model,
  File "/home/pedro/miniconda3/lib/python3.8/site-packages/unet/trainer.py", line 96, in fit
    history = model.fit(train_dataset,
  File "/home/pedro/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1183, in fit
    tmp_logs = self.train_function(iterator)
  File "/home/pedro/miniconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "/home/pedro/miniconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 917, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "/home/pedro/miniconda3/lib/python3.8/s

TypeError: object of type 'NoneType' has no len()

In [ ]:
x = list(train_dataset.take(1))

In [51]:
a, b = x[0]

In [52]:
unet_model(a[None, :, :, :]).shape

TensorShape([1, 64, 472, 2])

In [54]:
(107-64)//2

21

In [48]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
dataset = dataset.map(lambda x: x*2)
dataset = dataset.map(lambda x: x*2)
list(dataset.as_numpy_iterator())


[4, 8, 12]